In [110]:
import cv2
import os
import numpy as np


In [120]:
# Image load (흑백)
img_dir = "C:\Workspace\DP_0\DIP4E_Global_BookImages"
#img_name = "edge-ramp.tif"
img_name = "checkerboard-noisy2.tif"
#img_name = "septagon-small.tif"
image = cv2.imread(os.path.join(img_dir, img_name), cv2.IMREAD_GRAYSCALE)

# cv2.imshow('Gaussian', bi)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [112]:
# Basic Global Thresholding
# 임의의 임계값을 기준으로 그룹을 나누어 평균을 구해 새로운 임계값 도출, 반복

T = 255//3
k = 0
bi = cv2.threshold(image, T, 255, cv2.THRESH_BINARY)[1]

cv2.imwrite('Basic_0_img.jpg', bi)

for i in range (10):
    mean_values = []
    binary_image = cv2.threshold(image, T, 255, cv2.THRESH_BINARY)[1]
    for group in np.unique(binary_image):
        group_pixels = image[binary_image == group]
        mean_value = np.mean(group_pixels)
        mean_values.append(mean_value)

    T = (mean_values[0]+mean_values[1])/2

    print('각 그룹의 평균 값:', mean_values)
    print(f'새로운 {i+1}번째 T값:', T)

    if k == T : 
        print('반복횟수:', i+1)
        print('최적의 T값:', T)
        break
    else:
        k = T

cv2.imwrite('Basic_1_img.jpg', binary_image)

각 그룹의 평균 값: [10.144371206900384, 211.65317546704577]
새로운 1번째 T값: 110.89877333697308
각 그룹의 평균 값: [10.3592854212178, 211.91525923781742]
새로운 2번째 T값: 111.13727232951761
각 그룹의 평균 값: [10.377696226530148, 211.93380064473385]
새로운 3번째 T값: 111.155748435632
각 그룹의 평균 값: [10.377696226530148, 211.93380064473385]
새로운 4번째 T값: 111.155748435632
반복횟수: 4
최적의 T값: 111.155748435632


True

In [113]:
# Optimum Global Thresholding Using Otsu's Method
# 옷수

def otsu_threshold(image, mask):
    # 히스토그램 계산
    hist = cv2.calcHist([image], [0], mask, [256], [0, 256])

    # 전체 픽셀 수
    total_pixels = image.shape[0] * image.shape[1]

    # 초기화
    sum_total = 0
    sum_background = 0
    weight_total = 0
    max_variance = 0
    threshold = 0

    for i in range(256):
        weight_total += hist[i] / total_pixels
        sum_total += i * (hist[i] / total_pixels)

    for i in range(256):
        weight_background = weight_total - hist[i] / total_pixels
        weight_foreground = hist[i] / total_pixels

        if weight_background == 0 or weight_foreground == 0:
            continue

        sum_background += i * (hist[i] / total_pixels)
        mean_background = sum_background / weight_background
        mean_foreground = (sum_total - sum_background) / weight_foreground

        variance_between = weight_background * weight_foreground * (mean_background - mean_foreground) ** 2

        if variance_between > max_variance:
            max_variance = variance_between
            threshold = i

    # 이미지 이진화
    binary_image = np.zeros_like(image)
    binary_image[image > threshold] = 255

    return binary_image, threshold

# Otsu's method를 이용한 이진화
binary_image, threshold = otsu_threshold(image, mask=None)
print(threshold)

# 결과 출력
cv2.imwrite('Otsu_img.jpg', binary_image)
# cv2.imshow('Original Image', image)
# cv2.imshow('Otsu Threshold', binary_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


89


True

In [114]:
# Using Img Smoothing to Improve Global Thresholding
# 더 많은 노이즈가 낀 이미지를 이진화 하기위해 블러 처리 후 Otsu 진행

# 이미지 스무딩을 위한 평균 필터 적용
window_size = 7
smoothed_image = cv2.blur(image, (window_size, window_size))

binary_image, threshold = otsu_threshold(smoothed_image, mask=None)
print(threshold)

# 결과 출력
cv2.imwrite('Smooth_img.jpg', binary_image)
# cv2.imshow('sm', smoothed_image)
# cv2.imshow('Thresholded Image', thresholded_image)
# cv2.imshow('Original Image', image)
# cv2.imshow('Thresholded Image2', binary_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


21


True

In [115]:
# Using edges to Improve Global Thresholding

def calculate_edge_image(image, th, method):
    if method == 'gradient':
        # 그라데이션 계산
        gradient_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
        gradient_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
        edge_image = np.sqrt(np.square(gradient_x) + np.square(gradient_y))

    elif method == 'laplacian':
        # 라플라시안 계산
        laplacian = cv2.Laplacian(image, cv2.CV_64F)
        edge_image = np.abs(laplacian)

    else:
        raise ValueError("Invalid method. Choose either 'gradient' or 'laplacian'.")

    edge_image = cv2.threshold(edge_image, edge_th, 255, cv2.THRESH_BINARY)[1]

    return edge_image

edge_method = 'gradient'  # 에지 계산 방법 선택 ('gradient' 또는 'laplacian')
edge_th = 127
edge_image = calculate_edge_image(image, edge_th, method=edge_method)
# 마스크 지정
mask = np.ones_like(edge_image, dtype=np.uint8) * 255

# Otsu
binary_image, threshold = otsu_threshold(image, mask)
print(threshold)

# 결과 출력
cv2.imwrite('edges_img.jpg', binary_image)
# cv2.imshow('sm', edge_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


89


True

In [130]:
# Multiple Thresholding
# 임계값 2개로 흰 회 검 나누기
# 현재 작동 x ㅋㅋ

def otsu_threshold3(image):

    # Otsu의 이진화 적용
    thresholded_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

    # 삼진화를 위한 임계값 구하기
    hist = cv2.calcHist([image], [0], None, [256], [0, 256])
    hist = cv2.convertScaleAbs(hist)
    thresholds = cv2.threshold(hist, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    threshold1 = int(thresholds[0])
    threshold2 = int(thresholds[1])

    # 삼진화 이미지 생성
    binary_image = np.zeros_like(image)
    binary_image[image > threshold2] = 255
    binary_image[(image > threshold1) & (image <= threshold2)] = 128

    return binary_image, thresholds

# 이미지 삼진화
result, th = otsu_threshold3(image)

print(th[0])
print(th[1])

# 결과 출력
cv2.imshow('Result', result)
cv2.waitKey(0)
cv2.destroyAllWindows()


[255]
[255]


In [117]:
# Variable Thresholding

#1. Image partitioning
# img_name = "septagon.tif"
# image = cv2.imread(os.path.join(img_dir, img_name), cv2.IMREAD_GRAYSCALE)
height, width = image.shape[:2]

top_left = image[:height//2, :width//2]
top_right = image[:height//2, width//2:]
bottom_left = image[height//2:, :width//2]
bottom_right = image[height//2:, width//2:]

mask = None
binary_image_TL, _ = otsu_threshold(top_left, mask)
binary_image_TR, _ = otsu_threshold(top_right, mask)
binary_image_BL, _ = otsu_threshold(bottom_left, mask)
binary_image_BR, _ = otsu_threshold(bottom_right, mask)

combined_image = np.vstack((np.hstack((binary_image_TL, binary_image_TR)), np.hstack((binary_image_BL, binary_image_BR))))

binary_image, _ = otsu_threshold(image, mask)

# cv2.imwrite('nonparti_img.jpg', binary_image)
cv2.imwrite('partition_img.jpg', combined_image)


True

In [ ]:
# Variable Thresholding

#2. based on local image properties
# 멀티플의 연장



In [ ]:
# Variable Thresholding

#3. Using moving averages
# 2의 연장


In [118]:
# Multivariable Thresh
# 위에 것들의 결합? 아마도